In [982]:
import numpy as np
import pandas as pd
import nltk
import re

df_train = pd.read_csv("train.csv")
a = df_train[df_train["class"] == 1].sample(3500)

b = pd.concat([df_train[df_train["class"] == 0], a])

c = pd.concat([df_train[df_train["class"] == 2], b])

df_train = c

df_test = pd.read_csv("test.csv")



In [983]:
df_train["tweet"] = df_train["tweet"].str.lower() # lowercase
df_train["tweet"] = df_train["tweet"].str.replace(r"http\S+", '') # remove url
df_train["tweet"] = df_train["tweet"].str.replace(r'\s*rt\s*@[^:]*:*', '') #rt
df_train["tweet"] = df_train["tweet"].str.replace(r'\$\w*', '')


df_test["tweet"] = df_test["tweet"].str.lower() # lowercase
df_test["tweet"] = df_test["tweet"].str.replace(r"http\S+", '') # remove url
df_test["tweet"] = df_test["tweet"].str.replace(r'\s*rt\s*@[^:]*:*', '') #rt
df_test["tweet"] = df_test["tweet"].str.replace(r'\$\w*', '')



In [984]:
df_train["tweet"] = df_train["tweet"].apply(lambda x: nltk.tokenize.casual.remove_handles(x)) # remove tagged user
df_test["tweet"] = df_test["tweet"].apply(lambda x: nltk.tokenize.casual.remove_handles(x)) # remove tagged user


df_train["tweet"] = df_train["tweet"].str.replace(r'\d+(\.\d+)?', '') # remove number
df_train["tweet"] = df_train["tweet"].str.replace(r'[^\w\d\s]', '') # remove punctuation
#df_train["tweet"] = df_train["tweet"].str.replace(r'&#d+;', '') # remove emoji

#df_test["tweet"] = df_test["tweet"].str.replace(r'&#d+;', '') # remove emoji                                   
df_test["tweet"] = df_test["tweet"].str.replace(r'\d+(\.\d+)?', '') # remove number
df_test["tweet"] = df_test["tweet"].str.replace(r'[^\w\d\s]', '') # remove punctuation

In [985]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

df_train["tweet"] = df_train["tweet"].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
df_test["tweet"] = df_test["tweet"].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))



In [986]:
#some = ["im", "u", "get", "like", "lol", "bitch", "nigga", "niggas"]

some = ["im", "u"]

for i in stop_words:
    some.append(re.sub(r'[^\w\d\s]', '',i))
    
some = set(some)

df_train["tweet"] = df_train["tweet"].apply(lambda x: ' '.join(term for term in x.split() if term not in some))
df_test["tweet"] = df_test["tweet"].apply(lambda x: ' '.join(term for term in x.split() if term not in some))

In [987]:
from nltk.stem.snowball import SnowballStemmer

lemmatizer = SnowballStemmer("english")
df_train["tweet"] = df_train["tweet"].apply(lambda x: ' '.join(lemmatizer.stem(term) for term in x.split()))
df_test["tweet"] = df_test["tweet"].apply(lambda x: ' '.join(lemmatizer.stem(term) for term in x.split()))

In [999]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.tokenize import TweetTokenizer

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import VotingClassifier

tokenizer = TweetTokenizer().tokenize

X_train, X_test, y_train, y_test = train_test_split(df_train["tweet"], df_train["class"], test_size=0.3)


In [1000]:
tfidf_vectorizer = TfidfVectorizer(max_features=2000, tokenizer=tokenizer)
tfidf_vectorizer.fit(df_train["tweet"])


X_train = tfidf_vectorizer.transform(X_train).toarray()
X_test = tfidf_vectorizer.transform(X_test).toarray()


real_train = tfidf_vectorizer.transform(df_train["tweet"]).toarray()
real_test = tfidf_vectorizer.transform(df_test["tweet"]).toarray()

#model = LGBMClassifier(max_depth=10, num_leaves=25)
model = LinearSVC(loss="hinge")
model.fit(real_train, df_train["class"])
pp = model.predict(real_test)

'''
param_grid = {"num_leaves": [10,15,20,25,30], "max_depth": [5,10,15,20,30]}
grid = GridSearchCV(LGBMClassifier(), param_grid=param_grid, verbose=3)
grid.fit(X_train, y_train)
grid.predict(X_test)
print(grid.best_score_)
print(grid.best_params_)

names = ["K near", "Decision tree", "random forest", "SGD", "naive bayes", "linear SVC"]
classifiers = [KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(),
               SGDClassifier(max_iter = 100), MultinomialNB(), LinearSVC(loss='hinge')]

models = [("K near", KNeighborsClassifier()), ("Decision tree", DecisionTreeClassifier()),
          ("random forest", RandomForestClassifier()), ("SGD", SGDClassifier(max_iter=100)),
          ("naive bayes", MultinomialNB()), ("linear SVC", LinearSVC(loss='hinge'))]

ensemble = VotingClassifier(estimators = models, voting="hard", n_jobs=-1)
ensemble.fit(X_train, y_train)
pp = ensemble.predict(X_test)
'''
#accuracy_score(pp, y_test)



'\nparam_grid = {"num_leaves": [10,15,20,25,30], "max_depth": [5,10,15,20,30]}\ngrid = GridSearchCV(LGBMClassifier(), param_grid=param_grid, verbose=3)\ngrid.fit(X_train, y_train)\ngrid.predict(X_test)\nprint(grid.best_score_)\nprint(grid.best_params_)\n\nnames = ["K near", "Decision tree", "random forest", "SGD", "naive bayes", "linear SVC"]\nclassifiers = [KNeighborsClassifier(), DecisionTreeClassifier(), RandomForestClassifier(),\n               SGDClassifier(max_iter = 100), MultinomialNB(), LinearSVC(loss=\'hinge\')]\n\nmodels = [("K near", KNeighborsClassifier()), ("Decision tree", DecisionTreeClassifier()),\n          ("random forest", RandomForestClassifier()), ("SGD", SGDClassifier(max_iter=100)),\n          ("naive bayes", MultinomialNB()), ("linear SVC", LinearSVC(loss=\'hinge\'))]\n\nensemble = VotingClassifier(estimators = models, voting="hard", n_jobs=-1)\nensemble.fit(X_train, y_train)\npp = ensemble.predict(X_test)\n'

In [996]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pp))

             precision    recall  f1-score   support

          0       0.53      0.38      0.45       222
          1       0.87      0.87      0.87      1079
          2       0.85      0.92      0.89       763

avg / total       0.83      0.84      0.83      2064



In [1001]:
result_pd = pd.DataFrame({"id": df_test["id"], "class": pd.Series(pp)})
result_pd.head()

,class,id
0,1,0
1,2,1
2,1,2
3,1,3
4,1,4


In [1002]:
result_pd.to_csv("result.csv",columns=['id','class'],index=False)